## Imports

In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

The Piotroski Score is a financial scoring system used to evaluate the financial strength of a company based on its financial statements.
The Piotroski Score is based on nine criteria, grouped into three main categories: profitability, leverage/liquidity, and operating efficiency. Here are the nine criteria:

- **CR1** — net income for the current year; 1 if positive, else 0
- **CR2** — return on assets (ROA) in the current year; 1 if positive, else 0
- **CR3** — operating cash flow in the current year; 1 if positive, else 0
- **CR4** — cash flow from operations being greater than net income; 1 if positive, else 0
- **CR5** — decreased leverage compared with the previous year; 1 if negative (i.e., long-term debt has reduced), else 0
- **CR6** — more liquidity compared with the previous year; 1 if positive, else 0
- **CR7** — lack of dilution compared with the previous year; 1 if positive or 0 (no changes to shares), else 0 
- **CR8** — higher gross margin compared to the previous year; 1 if positive, else 0
- **CR9** — higher asset turnover ratio compared to the previous year; 1 if positive, else 0
- **Score** — summation of all the criteria


In [10]:
# Stock symbols
SYMBOLS = 'EREGL.IS,ISDMR.IS,KCAER.IS,CIMSA.IS,OYAKC.IS,BTCIM.IS,HEKTS.IS,GUBRF.IS,EGGUB.IS,ENJSA.IS,GWIND.IS,AKFYE.IS,AYDEM.IS,AKSEN.IS,ARCLK.IS,VESBE.IS,AKMGY.IS,OZKGY.IS,AKSGY.IS,EKGYO.IS,BIMAS.IS,MGROS.IS,SOKM.IS,THYAO.IS,PGSUS.IS,DESA.IS,PARSN.IS,TUPRS.IS,BOBET.IS,BRISA.IS'.split(',')
# Criteria + Score 
CRITERIA = 'CR1,CR2,CR3,CR4,CR5,CR6,CR7,CR8,CR9,Score'.split(',')

# Column Names
COLUMN_MAPPING_RAW={'Symbol': 'Symbol', 'Name': 'Name', 'CR1': 'Net Income', 'CR2': 'Return on Assets',
                'CR3': 'Op. Cash Flow', 'CR4': 'Quality of Earn', 'CR5': 'Long Term Debt',
                'CR6': 'Current Ratio', 'CR7': 'New Shares', 'CR8': 'Gross Margin', 'CR9': 'Asset TR'}

# Column names mapping for score data
COLUMN_MAPPING_SCORE = COLUMN_MAPPING_RAW | {'Score':'Score'}

In [11]:
def net_income(ticker):
    df = ticker.income_stmt
    return df.loc['Net Income'].iloc[0]

def roa(ticker):
    df = ticker.balance_sheet
    avg_assets = (df.loc['Total Assets'].iloc[0] + df.loc['Total Assets'].iloc[1])/2
    return round(net_income(ticker)/avg_assets, 2)

def ocf(ticker):
    """Calculate Operating Cash Flow using Free Cash Flow and Captial Expenditure.
    Take the absolute value for Captial Expenditure as yf returns as a negative number """
    df = ticker.cash_flow
    if 'Operating Cash Flow' in df.index:
        return df.loc['Operating Cash Flow'].iloc[0]
    else:
        return df.loc['Free Cash Flow'].iloc[0] + abs(df.loc['Capital Expenditure']).iloc[0]

def ltdebt(ticker):
    df = ticker.balance_sheet
    return (df.loc['Long Term Debt'].iloc[1] - df.loc['Long Term Debt'].iloc[0])

def current_ratio(ticker):
    df = ticker.balance_sheet
    current_ratio_current = df.loc['Total Assets'].iloc[0]/df.loc['Total Liabilities Net Minority Interest'].iloc[0]
    current_ratio_prev = df.loc['Total Assets'].iloc[1]/df.loc['Total Liabilities Net Minority Interest'].iloc[1]
    return round((current_ratio_current - current_ratio_prev), 2)

def new_shares(ticker):
    df = ticker.balance_sheet
    return (df.loc['Common Stock'].iloc[1] - df.loc['Common Stock'].iloc[0])

def gross_margin(ticker):
    df = ticker.income_stmt
    gross_margin_current = df.loc['Gross Profit'].iloc[0]/ df.loc['Total Revenue'].iloc[0]
    gross_margin_prev = df.loc['Gross Profit'].iloc[1]/ df.loc['Total Revenue'].iloc[1]
    return (gross_margin_current - gross_margin_prev)

def asset_turnover_ratio(ticker):
    df_bs = ticker.balance_sheet
    y0, y1, y2 = df_bs.loc['Total Assets'].iloc[0], df_bs.loc['Total Assets'].iloc[1], df_bs.loc['Total Assets'].iloc[2]
    avg_asset_y0 = (y0 + y1)/2
    avg_asset_y1 = (y1 + y2)/2
    
    df_is = ticker.income_stmt
    tot_rvn_y0 = df_is.loc['Total Revenue'].iloc[0]/avg_asset_y0
    tot_rvn_y1 = df_is.loc['Total Revenue'].iloc[1]/avg_asset_y1

    return round((tot_rvn_y0 - tot_rvn_y1), 2)


Store functions in a dictionary to call the relevant function in calculating the Piotroski score.

In [12]:
criteria_dict = {
    'CR1':net_income,
    'CR2':roa,
    'CR3':ocf,
    'CR5':ltdebt,
    'CR6':current_ratio,
    'CR7':new_shares,
    'CR8':gross_margin,
    'CR9':asset_turnover_ratio
}

In [13]:
def calculate_piotroski_score():
    #Loop through each symbol and retrieve ticker data using
    ps_criteria = {
        'Symbol':[],'Name':[],'CR1':[],'CR2':[],'CR3':[],'CR4':[],'CR5':[],
        'CR6':[],'CR7':[],'CR8':[],'CR9':[]
    }

    ps_criteria_data = {
        'Symbol':[],'Name':[],'CR1':[],'CR2':[],'CR3':[],'CR4':[],'CR5':[],
        'CR6':[],'CR7':[],'CR8':[],'CR9':[]
    }

    for symbol in SYMBOLS:
        ticker = yf.Ticker(symbol)


        ps_criteria['Symbol'].append(ticker.info['symbol'])
        ps_criteria['Name'].append(ticker.info['longName'])
        
        ps_criteria_data['Symbol'].append(ticker.info['symbol'])
        ps_criteria_data['Name'].append(ticker.info['longName'])

        #Loop through the dictionary containing criteria and their associated functions
        for key, value in criteria_dict.items():
            try:
                
                result = value(ticker)
                ps_criteria_data[key].append(result)
                #Special adjustment for CR7 - If the number of shares for the current year decreases or stays the same (0 is returned as the difference), a value of 1 is assigned.
                if key == 'CR7':
                    ps_criteria[key].append(1 if result >= 0 else 0)
                else:
                    #Process with other CRs
                    ps_criteria[key].append(1 if result > 0 else 0)
            except (KeyError, IndexError) as err:
                #Zero is assigned to the score and nan to raw data for any errors
                print(err)
                ps_criteria[key].append(0)
                ps_criteria_data[key].append(np.nan)

        #CR4 - Compute the score for CR4 using data for CR1 and CR3
        if ps_criteria_data['CR3'][-1] > ps_criteria_data['CR1'][-1]:
            ps_criteria['CR4'].append(1)
            ps_criteria_data['CR4'].append(1)
        else:
            #Set criteria and raw data to false (0)
            ps_criteria['CR4'].append(0)
            ps_criteria_data['CR4'].append(0)
    return ps_criteria, ps_criteria_data

Create DataFrames

In [14]:
ps_criteria, ps_criteria_data = calculate_piotroski_score()
ps_criteria_df = pd.DataFrame(ps_criteria)
# Add ranking scores to get the total score
ps_criteria_df['Score'] = ps_criteria_df[CRITERIA[:-1]].sum(axis=1)
ps_criteria_df

'Long Term Debt'
'Long Term Debt'
'Long Term Debt'


,Symbol,Name,CR1,CR2,CR3,CR4,CR5,CR6,CR7,CR8,CR9,Score
0,EREGL.IS,Eregli Demir ve Çelik Fabrikalari T.A.S.,1,1,1,1,0,0,1,0,0,5
1,ISDMR.IS,Iskenderun Demir ve Çelik A.S.,1,1,1,1,0,0,1,0,0,5
2,KCAER.IS,Kocaer Celik Sanayi ve Ticaret Anonim Sirketi,1,1,1,0,0,1,1,1,0,6
3,CIMSA.IS,Çimsa Çimento Sanayi ve Ticaret A.S.,1,1,1,1,0,1,0,1,0,6
4,OYAKC.IS,OYAK Çimento Fabrikalari A.S.,1,1,1,0,1,1,0,1,1,7
5,BTCIM.IS,Batiçim Bati Anadolu Çimento Sanayii Anonim Si...,1,1,1,0,1,1,1,1,0,7
6,HEKTS.IS,Hektas Ticaret T.A.S.,0,0,1,1,0,0,1,0,0,3
7,GUBRF.IS,Gübre Fabrikalari Türk Anonim Sirketi,0,0,1,1,0,1,1,0,0,4
8,EGGUB.IS,Ege Gübre Sanayii A.S.,1,0,1,0,0,0,0,0,0,2
9,ENJSA.IS,Enerjisa Enerji A.S.,1,1,1,1,0,1,1,1,0,7


In [15]:
ps_criteria_data_df = pd.DataFrame(ps_criteria_data)
ps_criteria_data_df

,Symbol,Name,CR1,CR2,CR3,CR4,CR5,CR6,CR7,CR8,CR9
0,EREGL.IS,Eregli Demir ve Çelik Fabrikalari T.A.S.,4.033089e+09,0.02,1.306814e+10,1,-2.486547e+09,-0.49,0.000000e+00,-0.103567,-0.24
1,ISDMR.IS,Iskenderun Demir ve Çelik A.S.,4.601547e+09,0.04,1.382081e+10,1,-4.553800e+08,-0.53,0.000000e+00,-0.080867,-0.11
2,KCAER.IS,Kocaer Celik Sanayi ve Ticaret Anonim Sirketi,1.186704e+09,0.10,5.536528e+08,0,-3.296917e+08,0.50,0.000000e+00,0.019936,-1.04
3,CIMSA.IS,Çimsa Çimento Sanayi ve Ticaret A.S.,2.490891e+09,0.07,2.836809e+09,1,-4.548940e+08,0.29,-8.105070e+08,0.082167,-0.05
4,OYAKC.IS,OYAK Çimento Fabrikalari A.S.,8.127896e+09,0.23,7.232825e+09,0,5.208406e+07,1.98,-8.678496e+07,0.056705,0.04
5,BTCIM.IS,Batiçim Bati Anadolu Çimento Sanayii Anonim Si...,1.833899e+09,0.10,6.251900e+08,0,5.250970e+08,0.73,0.000000e+00,0.057628,-0.42
6,HEKTS.IS,Hektas Ticaret T.A.S.,-8.782390e+08,-0.04,1.477970e+09,1,-6.858029e+08,-0.14,0.000000e+00,-0.167645,-0.47
7,GUBRF.IS,Gübre Fabrikalari Türk Anonim Sirketi,-4.172754e+08,-0.02,2.536379e+09,1,NaN,0.47,0.000000e+00,-0.041965,-0.15
8,EGGUB.IS,Ege Gübre Sanayii A.S.,1.001039e+09,NaN,4.719275e+08,0,NaN,NaN,NaN,-0.044462,NaN
9,ENJSA.IS,Enerjisa Enerji A.S.,4.517326e+09,0.05,6.470128e+09,1,-9.140351e+09,0.20,0.000000e+00,0.009453,-1.84


In [16]:
def make_pretty_score(styler):
    # Columns with 0 decimal points
    zero_formats = {}
    for idx in range(1,10):
        zero_formats[f'CR{idx}']='{:.0f}'        
    styler.format(zero_formats)
    
    # Hide index
    styler.hide(axis='index')

    # Left text alignment for Symbol and Name columns
    styler.set_properties(subset=['Symbol', 'Name'], **{'text-align': 'left'})
    # Center alignment for the rest
    styler.set_properties(subset=CRITERIA, **{'text-align': 'center'})

    # Set background gradients
    styler.background_gradient(subset=['Score'], cmap='PiYG')
    
    # Borders
    styler.set_properties(**{'border': '1px solid grey'})
    return styler

def column_formatter_score(name):
    return COLUMN_MAPPING_SCORE[name]

In [17]:
# Add table caption and styles to DF
ps_criteria_df.style.pipe(make_pretty_score).set_caption(f'Piotroski Score').set_table_styles(
    [{'selector': 'th.col_heading', 'props': [('text-align', 'center'),
                                              ('color', 'bisque'), ('font-size', '10pt')]},
     {'selector': 'td', 'props': [('font-size', '10pt')]},
     {'selector': 'caption', 'props': [('text-align', 'center'), ('color', 'goldenrod'),
                                       ('font-size', '14pt'), ('font-weight', 'bold')]}]).format_index(column_formatter_score, axis=1) 

Symbol,Name,Net Income,Return on Assets,Op. Cash Flow,Quality of Earn,Long Term Debt,Current Ratio,New Shares,Gross Margin,Asset TR,Score
EREGL.IS,Eregli Demir ve Çelik Fabrikalari T.A.S.,1,1,1,1,0,0,1,0,0,5
ISDMR.IS,Iskenderun Demir ve Çelik A.S.,1,1,1,1,0,0,1,0,0,5
KCAER.IS,Kocaer Celik Sanayi ve Ticaret Anonim Sirketi,1,1,1,0,0,1,1,1,0,6
CIMSA.IS,Çimsa Çimento Sanayi ve Ticaret A.S.,1,1,1,1,0,1,0,1,0,6
OYAKC.IS,OYAK Çimento Fabrikalari A.S.,1,1,1,0,1,1,0,1,1,7
BTCIM.IS,Batiçim Bati Anadolu Çimento Sanayii Anonim Sirketi,1,1,1,0,1,1,1,1,0,7
HEKTS.IS,Hektas Ticaret T.A.S.,0,0,1,1,0,0,1,0,0,3
GUBRF.IS,Gübre Fabrikalari Türk Anonim Sirketi,0,0,1,1,0,1,1,0,0,4
EGGUB.IS,Ege Gübre Sanayii A.S.,1,0,1,0,0,0,0,0,0,2
ENJSA.IS,Enerjisa Enerji A.S.,1,1,1,1,0,1,1,1,0,7
